# chatbot "Korea Trip Planner"
"**Korea Trip Planner**" sets up a day-by-day trip to Korea

In [10]:
from chatbot.chatbot import Chatbot

In [11]:
import uuid
import time

In [12]:
import csv
import pandas as pd

## Read korea tourist information csv file

In [13]:
def read_tourism_spots(csv_file_path):
    with open(csv_file_path, 'r', encoding='utf-8') as file:
        csv_string = file.read()
    return csv_string

csv_file_path = './data/test_set.csv'

test_set_locations = read_tourism_spots(csv_file_path)

In [14]:
test_set_locations

'POI_ID,POI_NM,CTPRVN_NM,SIGNGU_NM,LC_LO,LC_LA\n20868054,용리단길,서울특별시,용산구,126.9715184,37.53140714\n16497619,오이도포구해양관광단지,경기도,시흥시,126.6882923,37.34802011\n16984778,송우가구거리,경기도,포천시,127.1263588,37.79171158\n11178201,족발골목,부산광역시,중구,129.0268061,35.09945931\n20868111,대신동꽃마을,부산광역시,서구,129.0071313,35.12755173\n21085600,녹산수문아울렛패션거리,부산광역시,강서구,128.8903692,35.11842346\n21085542,엄상마을문화거리,경기도,양주시,127.0682909,37.83117656\n16589482,모래내시장먹자골목,서울특별시,서대문구,126.9161613,37.56900814\n21085521,수원남문시장푸드트레일러,경기도,수원시 팔달구,127.0190164,37.27763653\n21086085,백운호수음식마을,경기도,의왕시,127.0006,37.38130249\n20988095,전포카페거리,부산광역시,부산진구,129.0634863,35.15539778\n21085896,오이도포장마차촌,경기도,시흥시,126.6862772,37.34549407\n17001477,동두천시외국인관광특구,경기도,동두천시,127.0561696,37.91326724\n16731257,세종마을음식문화거리,서울특별시,종로구,126.9720196,37.5765362\n18275764,송리단길,서울특별시,송파구,127.1093999,37.50986671\n17398983,솔밭예술마을,부산광역시,해운대구,129.1622703,35.16466929\n8292244,BIFF,부산광역시,중구,129.0280458,35.09828051\n6323554,불광동먹자골목,서울특별시,은평구,126.9305302,37.61032714\n21085614,백년송도골목,부산광역시,

In [15]:
print(test_set_locations)

POI_ID,POI_NM,CTPRVN_NM,SIGNGU_NM,LC_LO,LC_LA
20868054,용리단길,서울특별시,용산구,126.9715184,37.53140714
16497619,오이도포구해양관광단지,경기도,시흥시,126.6882923,37.34802011
16984778,송우가구거리,경기도,포천시,127.1263588,37.79171158
11178201,족발골목,부산광역시,중구,129.0268061,35.09945931
20868111,대신동꽃마을,부산광역시,서구,129.0071313,35.12755173
21085600,녹산수문아울렛패션거리,부산광역시,강서구,128.8903692,35.11842346
21085542,엄상마을문화거리,경기도,양주시,127.0682909,37.83117656
16589482,모래내시장먹자골목,서울특별시,서대문구,126.9161613,37.56900814
21085521,수원남문시장푸드트레일러,경기도,수원시 팔달구,127.0190164,37.27763653
21086085,백운호수음식마을,경기도,의왕시,127.0006,37.38130249
20988095,전포카페거리,부산광역시,부산진구,129.0634863,35.15539778
21085896,오이도포장마차촌,경기도,시흥시,126.6862772,37.34549407
17001477,동두천시외국인관광특구,경기도,동두천시,127.0561696,37.91326724
16731257,세종마을음식문화거리,서울특별시,종로구,126.9720196,37.5765362
18275764,송리단길,서울특별시,송파구,127.1093999,37.50986671
17398983,솔밭예술마을,부산광역시,해운대구,129.1622703,35.16466929
8292244,BIFF,부산광역시,중구,129.0280458,35.09828051
6323554,불광동먹자골목,서울특별시,은평구,126.9305302,37.61032714
21085614,백년송도골목,부산광역시,서구,129.021345,35.078

In [16]:
# role - system instance_context
type_role = "You are the best travel planner who can make efficient travel plans. You make plans that are more efficient and readable than anyone else."

# ? - system type_role
instance_context = """
See this desciption:\n{}.\nPlease write based on the description.
You are the best travel planner who can make efficient travel plans.
<p>When responding:</p>
<ul>
    <li>Please provide information on the travel route, tourist destination (POI_NM), and province (CTPRVN_NM).</li>
    <li>Please fill out three tourist attractions in Korean and English per day.</li>
    <li>Always incorporate emojis when apt. 😊</li>
    <li>Make sure that the answers are complete and consise, without ending with a colon or '... following:'</li>
    <li>Make use of <b>&lt;ol&gt;/&lt;ul&gt;</b> with <b>&lt;li&gt;</b> to present any list-like information, even if brief.</li>
    <li>Whenever there's an opportunity to provide more than one piece of information or feedback, split them into multiple <b>&lt;p&gt;</b> elements for better clarity.</li>
    <li>Always format responses using valid HTML: e.g., <b>&lt;p&gt;</b> for paragraphs, <b>&lt;ul&gt;/&lt;ol&gt;</b> with <b>&lt;li&gt;</b> for lists, and <b>&lt;b&gt;</b> for emphasis or tourist attractions.</li>
    <li>Maintain a nihilistic humorous tone. Keep it brief, but don't sacrifice clarity for brevity.</li>
    <li>Please organize your travel route efficiently considering latitude (LC_LA) longitude (LC_LO) and order the same CTPRVN_NM value.</li>
</ul>
<p>Example response</p>
Day 1:<br />
<ul>
    <li><b>서울숲은행나무길 (Seoul Forest Ginkgo Tree Road)</b> in Seoul</li>
    <li><b>청계천 (Cheonggyecheon Stream)</b> in Seoul</li>
    <li><b>명동거리 (Myeongdong Street)</b> in Seoul</li>
</ul>

Day 2:<br />
<ul>
    <li><b>남산공원남산케이블카 (Namsan Park Namsan Cable Car)</b> in Seoul</li>
    <li><b>북창동먹자골목 (Bukchang-dong Food Alley)</b> in Seoul</li>
    <li><b>한국인삼주마을 (Korean Ginseng Village)</b> in Gyeonggi Province</li>
</ul>

Day 3:<br />
<ul>
    <li><b>안성다문화마을특구 (Anseong Damunhwa Village Special District)</b> in Gyeonggi Province</li>
    <li><b>의정부부대찌개거리 (Uijeongbu Army Stew Alley)</b> in Gyeonggi Province</li>
    <li><b>용문산토속음식마을 (Yongmun Mountain Traditional Food Village)</b> in Gyeonggi Province</li>
</ul>
Enjoy your trip and make wonderful memories in Korea! 😊🇰🇷

Don't forget always using valid HTML tags espacially <br />
""".format(test_set_locations)

# user
# instance_starter = "Make a plan for a trip in Korea."
instance_starter = "As a Korea trip planner, write one very short message to use at the beginning of the conversation in English"

In [17]:

# Amount of instances to be created
number_of_instances = 1

# Change the following to a list of hardcoded instance IDs if you want to use existing users.
user_ids = [str(uuid.uuid4()) for _ in range(number_of_instances)]

c  = 0 # counter for successful requests, don't change
error_c = 0 # counter for failed requests, don't change
for user_id in user_ids:
    bot = Chatbot(
        database_file="database/chatbot.db", 
        type_id="zhfldk82", # Korean Trip Planner
        user_id=user_id,
        type_name="Korean Trip Planner",
        type_role=type_role,
        instance_context=instance_context,
        instance_starter=instance_starter
    )
    try:
        # each bot should have a first message to the user
        print(bot.start())
    except:
        error_c += 1
        continue
    c+=1
    time.sleep(15) #openai seems to produce more errors if we send the requests too fast.
    
print("successful: {}, failed: {}".format(c, error_c))


successful: 0, failed: 3


##### Obtain URLs of all instances of a type

In [ ]:
pythonanywhere_username = "sonahskku"
type_id = "zhfldk82" # Korean Trip Planner
bot = Chatbot(
    database_file="database/chatbot.db", 
    type_id=type_id,
    user_id=user_ids[0]
)

for user_id in bot.type_instances():
    print("https://{}.pythonanywhere.com/{}/{}/chat".format(pythonanywhere_username, type_id, user_id))

v1  
http://127.0.0.1:5000/zhfldk82/795607ba-d790-4acc-8583-4ee1b7bac10c/chat